**

**First of all need to import all the libraries**

In [ ]:
# Python Package imports
import requests
from bs4 import BeautifulSoup
from dateutil.parser import parse
import concurrent.futures
import pandas as pd

**We want the IMDB data of top 50.So we set their threada upto 50**

In [ ]:
# Maximum number of threads that will be spawned
max_data = 50

**Now creating the variables which stores the values of movies title,year,genere,synopis,url of the movie and it's image id**

In [ ]:
movie_title_arr = []
movie_year_arr = []
movie_genre_arr = []
movie_synopsis_arr =[]
image_url_arr= []
image_id_arr = []

**After creating the variables we going to create different functions for scraping different data from different coulmns**

**Create a function for movies title**

In [ ]:
def getMovieTitle(header):
    try:
        return header[0].find("a").getText()
    except:
        return 'NA'

**Function for extracting the release date of all the movies**

In [ ]:
def getReleaseYear(header):
    try:
        return header[0].find("span",  {"class": "lister-item-year text-muted unbold"}).getText()
    except:
        return 'NA'

**Function for getting the genre of the movies**

In [ ]:
def getGenre(muted_text):
    try:
        return muted_text.find("span",  {"class":  "genre"}).getText()
    except:
        return 'NA'

**To get synopis of the movies**

In [ ]:

def getsynopsys(movie):
    try:
        return movie.find_all("p", {"class":  "text-muted"})[1].getText()
    except:
        return 'NA'

**Create function to extract image url**

In [ ]:
def getImage(image):
    try:
        return image.get('loadlate')
    except:
        return 'NA'

**Lastly creating a function for image id or image icon of the movie**

In [ ]:
def getImageId(image):
    try:
        return image.get('data-tconst')
    except:
        return 'NA'

**After that we creating a main function where we call all the above functions and also extract the data with the functions to obtain a list of top 50 IMDB movies data** 

In [ ]:
def main(imdb_url):
    response = requests.get(imdb_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Movie Name
    movies_list  = soup.find_all("div", {"class": "lister-item mode-advanced"})
    
    for movie in movies_list:
        header = movie.find_all("h3", {"class":  "lister-item-header"})
        muted_text = movie.find_all("p", {"class":  "text-muted"})[0]
        imageDiv =  movie.find("div", {"class": "lister-item-image float-left"})
        image = imageDiv.find("img", "loadlate")
        
        #  Movie Title
        movie_title =  getMovieTitle(header)
        movie_title_arr.append(movie_title)
        
        #  Movie release year
        year = getReleaseYear(header)
        movie_year_arr.append(year)
        
        #  Genre  of movie
        genre = getGenre(muted_text)
        movie_genre_arr.append(genre)
        
        # Movie Synopsys
        synopsis = getsynopsys(movie)
        movie_synopsis_arr.append(synopsis)
        
        #  Image attributes
        img_url = getImage(image)
        image_url_arr.append(img_url)
        
        image_id = image.get('data-tconst')
        image_id_arr.append(image_id)

**After that we running a loop to scrap starting 50 movies data from the imdb store**

In [ ]:
# An array to store all the URL that are being queried
image_array = []

# Maximum number of pages one wants to iterate over
max_pg =51

# Loop to generate all the URLS.
for i in range(0,max_pg):
    total_records = 0 if i==0 else (250*i)+1
    print(total_records)
    imdb_url = f'https://www.imdb.com/search/title/?release_date=2020-01-02,2021-02-01&user_rating=4.0,10.0&languages=en&count=250&start={total_records}&ref_=adv_nxt'
    image_array.append(imdb_url)

0
251
501
751
1001
1251
1501
1751
2001
2251
2501
2751
3001
3251
3501
3751
4001
4251
4501
4751
5001
5251
5501
5751
6001
6251
6501
6751
7001
7251
7501
7751
8001
8251
8501
8751
9001
9251
9501
9751
10001
10251
10501
10751
11001
11251
11501
11751
12001
12251
12501


In [ ]:
def download_stories(story_urls):
    data_list = min(max_data, len(story_urls))
    with concurrent.futures.ThreadPoolExecutor(max_workers=data_list) as executor:
        executor.map(main, story_urls)

**Now after scraping the data we going to save the data output in the csv file**

In [ ]:
# Call the download function with the array of URLS called imageArr
download_stories(image_array)

# Attach all the data to the pandas dataframe. You can optionally write it to a CSV file as well
movieDf = pd.DataFrame({
    "Title": movie_title_arr,
    "Release_Year": movie_year_arr,
    "Genre": movie_genre_arr,
    "Synopsis": movie_synopsis_arr,
    "image_url": image_url_arr,
    "image_id": image_id_arr,
})

print('--------- Download Complete CSV Formed --------')

movieDf.to_csv('file.csv', index=False)

movieDf.head()

--------- Download Complete CSV Formed --------


,Title,Release_Year,Genre,Synopsis,image_url,image_id
0,Ted Lasso,(2020– ),"\nComedy, Drama, Sport",\nAmerican college football coach Ted Lasso he...,https://m.media-amazon.com/images/M/MV5BMDVmOD...,tt10986410
1,Warrior Nun,(2020– ),"\nAction, Drama, Fantasy","\nAfter waking up in a morgue, an orphaned tee...",https://m.media-amazon.com/images/M/MV5BNjI1NT...,tt9059350
2,Big Sky,(2020– ),"\nCrime, Drama, Mystery",\nA private detective teams up with an ex-cop ...,https://m.media-amazon.com/images/M/MV5BYTM2Y2...,tt11794642
3,Avenue 5,(2020– ),"\nComedy, Sci-Fi","\nThe troubled crew of Avenue 5, a space cruis...",https://m.media-amazon.com/images/M/MV5BZGMyMz...,tt10234362
4,Gangs of London,(2020– ),"\nAction, Crime, Drama",\nTells the story of London being torn apart b...,https://m.media-amazon.com/images/M/MV5BOGJlZT...,tt7661390
